In [ ]:
import pandas as pd
import numpy as np

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip -q install geopandas pyogrio shapely

In [ ]:
RAW_DIR = "/content/drive/MyDrive/CIS_671_datasets"
GV_COUNTY_CSV = os.path.join(RAW_DIR, "Medicare_Geo_Variation_County.csv")
HOSPITALS_CSV = os.path.join(RAW_DIR, "Hospital_General_Information.csv")
MSDRG_CSV     = os.path.join(RAW_DIR, "Medicare_Inpatient_Hospitals_by_Provider_and_Service.csv")
COUNTY_SHP    = os.path.join(RAW_DIR, "geo", "cb_2023_us_county_5m.shp")

hosp = pd.read_csv(HOSPITALS_CSV, dtype=str, low_memory=False)
list(hosp.columns)

In [ ]:
gv = pd.read_csv(GV_COUNTY_CSV, dtype=str, low_memory=False)
list(gv.columns)

In [ ]:
drg = pd.read_csv(MSDRG_CSV, encoding="cp1252", engine="python")
list(drg.columns)

In [ ]:
import os, re, json, math
import numpy as np
import pandas as pd
import geopandas as gpd


RAW_DIR = "/content/drive/MyDrive/CIS_671_datasets"
GV_COUNTY_CSV = os.path.join(RAW_DIR, "Medicare_Geo_Variation_County.csv")
HOSPITALS_CSV = os.path.join(RAW_DIR, "Hospital_General_Information.csv")
MSDRG_CSV     = os.path.join(RAW_DIR, "Medicare_Inpatient_Hospitals_by_Provider_and_Service.csv")
COUNTY_SHP    = os.path.join(RAW_DIR, "geo", "cb_2023_us_county_5m.shp")

TARGET_YEAR_FOR_SPEND = "2023"
MI_STATE_FIPS = "26"             # Michigan
OUT_DIR = os.path.join(RAW_DIR, "data_tidy")
os.makedirs(OUT_DIR, exist_ok=True)


def pick_col_name(df: pd.DataFrame, candidates):
    """Return the first existing column NAME from candidates (or None)."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def std_z(x: pd.Series):
    mu, sd = x.mean(), x.std(ddof=0)
    if sd == 0 or np.isnan(sd):
        return pd.Series([0]*len(x), index=x.index)
    return (x - mu) / sd

def clean_county_name(s):
    if pd.isna(s): return s
    s = str(s).strip()
    s = re.sub(r"\s+County$", "", s, flags=re.I)
    s = re.sub(r"\s+Parish$", "", s, flags=re.I)
    s = re.sub(r"\s+City and Borough$|\s+Borough$|\s+City$", "", s, flags=re.I)
    return s.upper()

def classify_med_surg(desc: str):
    if not isinstance(desc, str): return "Medical"
    d = desc.upper()
    surg_kw = ["SURG","OPER","OPERATION","PROCED","REPAIR","EXCISION","RESECTION","TRANSPL","CRANIOTOMY"]
    return "Surgical" if any(k in d for k in surg_kw) else "Medical"


# County shapefile → FIPS crosswalk
g = gpd.read_file(COUNTY_SHP)[["STATEFP","COUNTYFP","GEOID","NAME","STUSPS","geometry"]].copy()
g["COUNTY_NORM"] = g["NAME"].str.upper()

# crosswalk for joins (no geometry needed here)
xwalk = g[["STUSPS","COUNTY_NORM","GEOID"]].rename(
    columns={"STUSPS":"state", "COUNTY_NORM":"county_norm", "GEOID":"fips"}
)

# canonical labels for later (name/state)
meta = g[["GEOID","NAME","STUSPS"]].rename(
    columns={"GEOID":"fips","NAME":"county_name","STUSPS":"state"}
)

# Hospital roster (stars, beds, FIPS match)
hosp = pd.read_csv(HOSPITALS_CSV, dtype=str, low_memory=False)

# flexible column picks
col_id     = "Facility ID" if "Facility ID" in hosp.columns else pick_col_name(hosp, ["Provider ID","Provider ID Number"])
col_name   = pick_col_name(hosp, ["Facility Name","Hospital Name"])
col_state  = pick_col_name(hosp, ["State"])
col_county = pick_col_name(hosp, ["County/Parish","County Name","County"])
col_stars  = pick_col_name(hosp, ["Hospital overall rating","Hospital Overall Rating","Hospital overall rating footnote"])
col_beds   = pick_col_name(hosp, ["Number of Certified Beds","Number of Certified Beds (Facility)","Beds"])
col_owner  = pick_col_name(hosp, ["Hospital Ownership","Hospital Type","Ownership"])

hosp_df = pd.DataFrame({
    "provider_id": hosp[col_id].astype(str).str.strip(),
    "name":        hosp[col_name].astype(str).str.strip(),
    "state":       hosp[col_state].astype(str).str.strip(),
    "county_raw":  hosp[col_county].astype(str).str.strip(),
    "stars":       pd.to_numeric(hosp[col_stars], errors="coerce") if col_stars else np.nan,
    "beds":        pd.to_numeric(hosp[col_beds],  errors="coerce") if col_beds  else np.nan,
    "ownership":   (hosp[col_owner].astype(str).str.strip() if col_owner else pd.Series([np.nan]*len(hosp)))
})

hosp_df["county_norm"] = hosp_df["county_raw"].map(clean_county_name)
hosp_df = hosp_df.merge(xwalk, left_on=["state","county_norm"], right_on=["state","county_norm"], how="left")
hosp_df["has_fips"] = hosp_df["fips"].notna()


# County quality = bed-weighted star rating
h = hosp_df.loc[hosp_df["fips"].notna(), ["fips","beds","stars"]].copy()
h["beds_eff"]  = pd.to_numeric(h["beds"], errors="coerce").fillna(0).clip(lower=0)
h["stars_eff"] = pd.to_numeric(h["stars"], errors="coerce")

h["ws"] = h["stars_eff"] * h["beds_eff"]
agg = h.groupby("fips", dropna=True, as_index=False).agg(
    beds_sum=("beds_eff","sum"),
    ws_sum=("ws","sum")
)
agg["quality_bedweighted"] = np.where(agg["beds_sum"]>0, agg["ws_sum"]/agg["beds_sum"], np.nan)
grp = agg[["fips","quality_bedweighted"]].copy()


# Geographic Variation (County spend per beneficiary) — use FIPS directly
gv = pd.read_csv(GV_COUNTY_CSV, dtype=str, low_memory=False)

col_year   = "YEAR"
col_level  = "BENE_GEO_LVL"
col_code   = "BENE_GEO_CD"

# prefer standardized per-capita
if "TOT_MDCR_STDZD_PYMT_PC" in gv.columns:
    spend_col = "TOT_MDCR_STDZD_PYMT_PC"
elif "TOT_MDCR_PYMT_PC" in gv.columns:
    spend_col = "TOT_MDCR_PYMT_PC"
elif "TOT_MDCR_STDZD_PYMT_AMT" in gv.columns and "BENES_TOTAL_CNT" in gv.columns:
    spend_col = "__computed_spend__"
    gv[spend_col] = pd.to_numeric(gv["TOT_MDCR_STDZD_PYMT_AMT"], errors="coerce") / \
                    pd.to_numeric(gv["BENES_TOTAL_CNT"], errors="coerce")
elif "TOT_MDCR_PYMT_AMT" in gv.columns and "BENES_TOTAL_CNT" in gv.columns:
    spend_col = "__computed_spend__"
    gv[spend_col] = pd.to_numeric(gv["TOT_MDCR_PYMT_AMT"], errors="coerce") / \
                    pd.to_numeric(gv["BENES_TOTAL_CNT"], errors="coerce")
else:
    raise ValueError("Could not find a per-beneficiary spend column in GV file.")

gv_year = gv[gv[col_year] == TARGET_YEAR_FOR_SPEND].copy()
gv_cnty = gv_year[gv_year[col_level].str.contains("county", case=False, na=False)].copy()

gv_cnty["fips"] = gv_cnty[col_code].astype(str).str.strip().str.zfill(5)
gv_cnty["spend"] = pd.to_numeric(gv_cnty[spend_col], errors="coerce")

gv_agg = gv_cnty.groupby("fips", as_index=False)["spend"].mean()


# Join spend + quality → z-scores, CSV
xwalk_nostate = xwalk.drop(columns=["state"], errors="ignore")

county = (
    xwalk_nostate
    .merge(gv_agg, on="fips", how="left")
    .merge(grp,     on="fips", how="left")
    .merge(meta,    on="fips", how="left")   # adds canonical state + county_name once
)

county["z_spend"]   = std_z(county["spend"])
county["z_quality"] = std_z(county["quality_bedweighted"])

county_out = county[["fips","state","county_name","spend","quality_bedweighted","z_spend","z_quality"]].copy()
county_out.to_csv(os.path.join(OUT_DIR, "counties_2023.csv"), index=False)

# MS-DRG aggregation per hospital
def read_robust_csv(path, try_encodings=("utf-8","utf-8-sig","cp1252","latin-1"),
                    try_seps=(",", "|", "\t")):
    last_err = None
    for enc in try_encodings:
        for sep in try_seps:
            try:
                df = pd.read_csv(path, sep=sep, encoding=enc,
                                 encoding_errors="replace",
                                 engine="python")
                return df, enc, sep
            except Exception as e:
                last_err = e
    raise RuntimeError(f"Failed to read {path}: {last_err}")

drg, used_enc, used_sep = read_robust_csv(MSDRG_CSV)
print("MSDRG loaded with:", used_enc, "separator:", repr(used_sep))

# ['Rndrng_Prvdr_CCN','DRG_Cd','DRG_Desc','Tot_Dschrgs','Avg_Mdcr_Pymt_Amt', ...]
col_pid   = pick_col_name(drg, ["Rndrng_Prvdr_CCN","Rndrng_Provider_CCN","Provider Id","Provider Number","Facility ID"])
col_drg   = pick_col_name(drg, ["DRG_Cd","MSDRG Code","DRG Definition Code"])
col_desc  = pick_col_name(drg, ["DRG_Desc","DRG Definition","MSDRG Description"])
col_disch = pick_col_name(drg, ["Tot_Dschrgs","Total Discharges","Discharges"])
col_avg_m = pick_col_name(drg, ["Avg_Mdcr_Pymt_Amt","Average Medicare Payments","Avg Medicare Payment Amount"])

for nm, label in [(col_pid,"provider id"),(col_drg,"drg code"),(col_desc,"drg desc"),
                  (col_disch,"discharges"),(col_avg_m,"avg medicare payment")]:
    if nm is None:
        raise ValueError(f"Missing expected MS-DRG column for {label}. "
                         f"Available: {list(drg.columns)[:30]}")

drg_df = pd.DataFrame({
    "provider_id": drg[col_pid].astype(str).str.strip(),
    "drg_code":    drg[col_drg].astype(str).str.zfill(3),
    "drg_desc":    drg[col_desc].astype(str),
    "discharges":  pd.to_numeric(drg[col_disch], errors="coerce").fillna(0).astype(int),
    "avg_medicare_payment": pd.to_numeric(drg[col_avg_m], errors="coerce")
})

# totals per hospital
tot = (drg_df.groupby("provider_id", as_index=False)["discharges"]
       .sum().rename(columns={"discharges":"total_discharges"}))
drg_df = drg_df.merge(tot, on="provider_id", how="left")

# shares + profile via vectorized agg
drg_df["share"]    = drg_df["discharges"] / drg_df["total_discharges"].replace(0, np.nan)
drg_df["share_sq"] = drg_df["share"].fillna(0)**2
drg_df["pay_w"]    = drg_df["avg_medicare_payment"] * drg_df["discharges"]

drg_profile = (drg_df.groupby("provider_id", as_index=False)
               .agg(hhi=("share_sq","sum"), pay_w_sum=("pay_w","sum"))
               .merge(tot, on="provider_id", how="left"))
drg_profile["wavg_payment"] = drg_profile["pay_w_sum"] / drg_profile["total_discharges"].replace(0, np.nan)
drg_profile = drg_profile.drop(columns=["pay_w_sum"])

# med vs surg mix
drg_df["category"] = drg_df["drg_desc"].map(classify_med_surg)
mix = (drg_df.pivot_table(index="provider_id", columns="category",
                          values="discharges", aggfunc="sum").fillna(0))
mix["med_share"]  = mix.get("Medical",0)  / (mix.get("Medical",0)+mix.get("Surgical",0)).replace(0, np.nan)
mix["surg_share"] = mix.get("Surgical",0) / (mix.get("Medical",0)+mix.get("Surgical",0)).replace(0, np.nan)
mix = mix[["med_share","surg_share"]].reset_index()

# top-5 DRGs per hospital
top5_rows = []
for pid, d in drg_df.groupby("provider_id"):
    d = d.sort_values("share", ascending=False).head(5)
    for rank, r in enumerate(d.itertuples(index=False), start=1):
        top5_rows.append({
            "provider_id": pid,
            "rank": rank,
            "drg_code": r.drg_code,
            "drg_desc": r.drg_desc,
            "share": float(r.share) if pd.notna(r.share) else 0.0,
            "avg_medicare_payment": float(r.avg_medicare_payment) if pd.notna(r.avg_medicare_payment) else np.nan
        })
top5 = pd.DataFrame(top5_rows)
top5.to_csv(os.path.join(OUT_DIR, "hospital_drg_top5.csv"), index=False)


# Hospitals table + DRG profile → CSV
hosp_profile = hosp_df.merge(drg_profile, on="provider_id", how="left").merge(mix, on="provider_id", how="left")
hosp_out = hosp_profile[[
    "provider_id","name","state","fips","stars","beds","ownership","hhi","wavg_payment","med_share","surg_share"
]]
hosp_out.to_csv(os.path.join(OUT_DIR, "hospitals_2025.csv"), index=False)


# Michigan county GeoJSON (for map)
g_mi = g[g["STATEFP"] == MI_STATE_FIPS].copy()
g_mi = g_mi.rename(columns={"GEOID":"fips","NAME":"county_name","STUSPS":"state"})
g_mi = g_mi[["fips","county_name","state","geometry"]]
g_mi.to_file(os.path.join(OUT_DIR, "counties_geo.json"), driver="GeoJSON")


# Multi-state (region) export + county ranking columns
#   26 = MI, 39 = OH, 18 = IN, 17 = IL, 55 = WI
THREE_STATES = ["26","39","18"]                 # MI, OH, IN
FIVE_STATES  = ["26","39","18","17","55"]       # MI, OH, IN, IL, WI
STATES = FIVE_STATES


# Region GeoJSON
g_region = g[g["STATEFP"].isin(STATES)].copy()
g_region = g_region.rename(columns={"GEOID":"fips","NAME":"county_name","STUSPS":"state"})
g_region = g_region[["fips","county_name","state","geometry"]]
g_region.to_file(os.path.join(OUT_DIR, "counties_geo_region.json"), driver="GeoJSON")

# County “capacity” metrics
# beds_sum: total certified beds in county
beds_by_cnty = (
    hosp_out.assign(beds=pd.to_numeric(hosp_out["beds"], errors="coerce"))
            .dropna(subset=["fips"])
            .groupby("fips", as_index=False)["beds"].sum()
            .rename(columns={"beds":"beds_sum"})
)

# discharges_sum: sum of hospital total discharges per county
if "total_discharges" not in drg_df.columns:
    _tot = (drg_df.groupby("provider_id", as_index=False)["discharges"]
                  .sum().rename(columns={"discharges":"total_discharges"}))
    drg_df = drg_df.merge(_tot, on="provider_id", how="left")

dis_by_cnty = (
    hosp_out[["provider_id","fips"]]
      .merge(drg_df[["provider_id","total_discharges"]].drop_duplicates("provider_id"),
             on="provider_id", how="left")
      .groupby("fips", as_index=False)["total_discharges"].sum()
      .rename(columns={"total_discharges":"discharges_sum"})
)

# Attach metrics to counties_2023.csv and rewrite
county_out = (county_out
              .merge(beds_by_cnty, on="fips", how="left")
              .merge(dis_by_cnty, on="fips", how="left"))

county_out["fips"] = county_out["fips"].astype(str).str.zfill(5)

county_out.to_csv(os.path.join(OUT_DIR, "counties_2023.csv"), index=False)

print("Wrote region GeoJSON + updated counties_2023 with beds_sum & discharges_sum.")

print("Done. Wrote:")
for f in ["counties_2023.csv","hospitals_2025.csv","hospital_drg_top5.csv","counties_geo.json"]:
    print(" -", os.path.join(OUT_DIR, f))

In [ ]:
import numpy as np
import pandas as pd

# Fix z-score helper to preserve NaNs rather than forcing 0
def std_z(x: pd.Series):
    x = pd.to_numeric(x, errors="coerce")
    if x.notna().sum() <= 1:
        return pd.Series([np.nan]*len(x), index=x.index)
    mu, sd = x.mean(), x.std(ddof=0)
    if sd == 0 or np.isnan(sd):
        return pd.Series([np.nan]*len(x), index=x.index)
    return (x - mu) / sd

# Recompute county z’s
county_out["z_spend"]   = std_z(county_out["spend"])
county_out["z_quality"] = std_z(county_out["quality_bedweighted"])

county_out["fips"] = county_out["fips"].astype(str).str.zfill(5)
hosp_out["fips"]   = hosp_out["fips"].astype(str).str.zfill(5)

# CCN / provider_id must be 6-char, zero-padded everywhere
pad6 = lambda s: s.astype(str).str.replace(r"\D","", regex=True).str.zfill(6)
hosp_out["provider_id"]   = pad6(hosp_out["provider_id"])
drg_profile["provider_id"] = pad6(drg_profile["provider_id"])
mix["provider_id"]         = pad6(mix["provider_id"])
top5["provider_id"]        = pad6(top5["provider_id"])

# Re-merge hospital profile so hhi / wavg_payment / shares populate
hosp_profile = (
    hosp_df.assign(provider_id=pad6(hosp_df["provider_id"]))
           .merge(drg_profile, on="provider_id", how="left")
           .merge(mix,        on="provider_id", how="left")
)

hosp_out = hosp_profile[[
    "provider_id","name","state","fips","stars","beds","ownership",
    "hhi","wavg_payment","med_share","surg_share"
]].copy()

county_out.to_csv(os.path.join(OUT_DIR, "counties_2023.csv"), index=False)
hosp_out.to_csv(os.path.join(OUT_DIR, "hospitals_2025.csv"), index=False)
top5.to_csv(os.path.join(OUT_DIR, "hospital_drg_top5.csv"), index=False)

print("Patched files rewritten in", OUT_DIR)
